## Clean original XML file

The original file is misformatted, let's clean that up

In [ ]:
from preprocessing.clean_original_xml import clean_original_xml
data_folder = '/home/daniel/s2ds/Data/'
raw_file = '01_full_output_cleaned.xml'
cleaned_file = '02_data.xml'
clean_original_xml(data_folder, raw_file, cleaned_file)

## Get client agent messages and full conversations
This part extract paired client-agent messages and full conversations (training data for Doc2Vec) as well, for more see docs.

In [12]:
import preprocessing.xml_parsing as xp
reload(xp)
data_folder = '/home/daniel/s2ds/Data/'
input_file = '02_data.xml'
# input_file = '02_data_test.xml'
client_messages_file = '03_clientMessages.txt'
agent_messages_file = '03_agentMessages.txt'
conv_messages_file = '03_doc2vecTrainingData.txt'
xp.get_client_agent_conversations(data_folder, input_file, client_messages_file, 
                                  agent_messages_file, conv_messages_file)

## Get training data for Word2Vec
This part is written so we can maximise the amount of training data we have for the Word2Vec model. See docs for more info.

In [ ]:
import preprocessing.xml_parsing as xp
reload(xp)
data_folder = '/home/daniel/s2ds/Data/'
input_file = '02_data.xml'
# input_file = '02_data_test.xml'
output_file = '03_word2vecTrainingData.txt'
xp.get_w2v_training(data_folder, input_file, output_file)

## Some quick line counting for sanity check

In [ ]:
%%script bash 
cd ~/s2ds/Data/
wc -l 03_clientMessages.txt
wc -l 03_agentMessages.txt
wc -l 03_doc2vecTrainingData.txt
wc -l 03_doc2vecTrainingData2.txt
wc -l 03_word2vecTrainingData.txt

## Filter word2vecTrainingData
Get rid of punctuation, names, emails postcodes, numbers, etc.. lot's going on, check documentation.

no n_grams, no lemmatization

In [ ]:
%cd ~/ms/Daniel/
import preprocessing.filtering as filtering
reload(filtering)
data_folder = '/home/daniel/s2ds/Data/'
input_file = '03_word2vecTrainingData.txt'
output_file = '04_word2vecTrainingDataFiltered.txt'

names = filtering.build_names(data_folder)
# word_cache is a dict with lemmatized values
word_cache = filtering.filter_corpus(data_folder, input_file, output_file, preprocessing=True, spellcheck=True,
                                     names=names, lemmatize=False)

## Filter doc2vecTrainingData2

no n_grams, no lemmatization

In [ ]:
%cd ~/ms/Daniel/
import preprocessing.filtering as filtering
reload(filtering)
data_folder = '/home/daniel/s2ds/Data/'
input_file = '03_doc2vecTrainingData2.txt'
output_file = '04_doc2vecTrainingDataFiltered2.txt'

names = filtering.build_names(data_folder)
# word_cache is a dict with lemmatized values
filtering.filter_corpus(data_folder, input_file, output_file, preprocessing=True, spellcheck=True,
                        names=names, lemmatize=False, nltk_preproc=word_cache)

## Make doc2vecTraingingData from  doc2vecTraingingData2
Just remove all ||.

In [ ]:
import codecs, os
import preprocessing.filtering as filtering
reload(filtering)
data_folder = '/home/daniel/s2ds/Data/'
input_file = '04_doc2vecTrainingDataFiltered2.txt'
output_file = '04_doc2vecTrainingDataFiltered.txt'
input_file = codecs.open(os.path.join(data_folder, input_file), 'rU', 'utf-8')
output_file = codecs.open(os.path.join(data_folder, output_file), 'w', 'utf-8')
for i, line in enumerate(input_file):
    output_file.write(line.replace('||', ''))
    # close files
input_file.close()
output_file.close()

## Filter client-agent message pairs
no n_grams, no lemmatization

In [ ]:
%cd ~/ms/Daniel/
import preprocessing.filtering as filtering
reload(filtering)
data_folder = '/home/daniel/s2ds/Data/'
client_file = '03_clientMessages.txt'
client_output = '04_clientMessagesFiltered.txt'
agent_file = '03_agentMessages.txt'
agent_output = '04_agentMessagesFiltered.txt'

names = filtering.build_names(data_folder)
filtering.filter_client_agent(data_folder, client_file, agent_file, client_output,
                              agent_output, preprocessing=True, spellcheck=True, names=names, lemmatize=False)

run client_agent_conv, to get a one2one correspondance between client messages and convNum for fastText training

In [6]:
%cd ~/ms/Daniel/
import preprocessing.filtering as filtering
reload(filtering)
data_folder = '/home/daniel/s2ds/Data/'
client_file = '03_clientMessages.txt'
client_output = '05_clientMessagesFilteredfastText.txt'
agent_file = '03_agentMessages.txt'
agent_output = '05_agentMessagesFilteredfastText.txt'
conv_output = '05_fastTextConv.txt'
names = filtering.build_names(data_folder)
filtering.filter_client_agent_conv(data_folder, client_file, agent_file, client_output, agent_output, 
                              conv_output, preprocessing=True, spellcheck=True, names=names, lemmatize=False)

/home/daniel/Dropbox/S2DS/Code/Daniel


## Check file lines after filtering

In [ ]:
%%script bash 
cd ~/s2ds/Data/
wc -l 04_clientMessagesFiltered.txt
wc -l 04_agentMessagesFiltered.txt
wc -l 04_word2vecTrainingDataFiltered.txt
wc -l 04_doc2vecTrainingDataFiltered.txt
wc -l 04_doc2vecTrainingDataFiltered2.txt
wc -l 05_clientMessagesFilteredfastText.txt
wc -l 05_agentMessagesFilteredfastText.txt
wc -l 05_fastTextConv.txt